In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import logging

logging.basicConfig(level=logging.INFO)

In [3]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [4]:
train=pd.read_csv('/kaggle/input/eh2week/train.csv')
test=pd.read_csv('/kaggle/input/eh2week/test.csv')
ss=pd.read_csv('/kaggle/input/eh2week/submission.csv')

In [5]:
%%time
import tensorflow_hub as hub 
import tokenization
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 5.05 s, sys: 1.15 s, total: 6.19 s
Wall time: 6.16 s


In [6]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(64, activation='tanh')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    #net = tf.keras.layers.Dense(480, activation='tanh')(net)
    #net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='tanh')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(3, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [7]:
%%time
max_len = 500
train_input = bert_encode(train.Review.values, tokenizer, max_len=max_len)
test_input = bert_encode(test.Review.values, tokenizer, max_len=max_len)
train_labels = tf.keras.utils.to_categorical(train.Sentiment.values, num_classes=3)

CPU times: user 2min 35s, sys: 489 ms, total: 2min 36s
Wall time: 2min 36s


In [8]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 500)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [10]:
%%time
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)

train_history = model.fit(
    train_input, train_labels, 
    validation_split=0.2,
    epochs=4,
    callbacks=[checkpoint, earlystopping],
    batch_size=10,
    verbose=1
)

Epoch 1/4
3528/3528 [==============================] - ETA: 0s - loss: 0.6749 - accuracy: 0.7120
Epoch 00001: val_accuracy improved from -inf to 0.74728, saving model to model.h5
3528/3528 [==============================] - 2667s 756ms/step - loss: 0.6749 - accuracy: 0.7120 - val_loss: 0.6080 - val_accuracy: 0.7473
Epoch 2/4
  64/3528 [..............................] - ETA: 39:37 - loss: 0.4891 - accuracy: 0.8234

KeyboardInterrupt: 

In [11]:
%%time 
model.load_weights('model.h5')
test_pred = model.predict(test_input)

CPU times: user 7min 4s, sys: 21.9 s, total: 7min 26s
Wall time: 6min 41s


In [12]:
ss1=pd.DataFrame(test_pred,columns=ss.columns)

In [13]:
ss1.to_csv('/kaggle/working/week16080.csv',index=False)